In [1]:
import pandas as pd
import os
import json
import jsonlines
from tqdm import tqdm
import glob
import yaml
from sqlalchemy import create_engine
from sqlalchemy.sql import text

with open("/home/ubuntu/work/therapeutic_accelerator/config/main.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
with open("../config/keys.yaml", "r") as f:
    keys = yaml.load(f, Loader=yaml.FullLoader)
    
bucket_path = os.path.join(config['paths']['root'], config['paths']['mount'])

ModuleNotFoundError: No module named 'jsonlines'

In [2]:
temp_df = pd.DataFrame()

file = "/home/ubuntu/work/bucket/fulltext-zipped/20230609_111759_00053_f77bs_0b8065d5-00f3-4eb0-8919-b403872088a4"

with jsonlines.open(file, "r") as reader: 
    count = 0
    for line in tqdm(reader.iter(skip_invalid=True)): 
        if count >= 5: 
            break
        else: 
            temp_df = pd.concat([temp_df,pd.DataFrame([line])]) # reads json, converts to dataframe, preprocess functions and appends results to database
            count += 1
            
temp_df.reset_index(inplace = True, drop = True)

0it [00:00, ?it/s]

5it [00:00,  7.50it/s]


Get section headers to split text into parts
|corpusID|authors|section(s)|
|---|---|---|

Don't need source, need text and annotations
- stored in lists with indexes convert to human readable

Get Corpus IDs from DB

In [10]:
# import psycopg2

# psy_engine = psycopg2.connect(
#     database=config['database']['name'],
#     user=config['database']['user'],
#     password=keys['postgres'],
#     host=config['database']['host'],
#     port='5432'
# )

# cur = psy_engine.cursor()

# conn = sql.connect()

# test = cur.execute("SELECT * FROM attributes;") 

# psy_engine.commit()
# cur.close()
# psy_engine.close()

In [4]:
# pull content out because that's all I need
test = temp_df.loc[0, 'content']
test.pop('source')

{'pdfurls': ['https://export.arxiv.org/pdf/2302.13777v2.pdf'],
 'pdfsha': '9a24b6ef9ec19eacbe7a223df8111f12d8da40ef',
 'oainfo': None}

In [ ]:
# turn into dataclass
def convert_text(text, key):
    temp = json.loads(text['annotations'][key])
    results = []
    for i in temp: 
        results.append(test['text'][temp['start']:temp['end']])
    return results

In [11]:
def get_abstract(text_dict):
    """ """
    ant_index = json.loads(text_dict['annotations']['abstract'])[0]
    ant = test['text'][ant_index['start']:ant_index['end']]
    return ant

In [12]:
get_abstract(test)

"A short X-ray burst was observed from the radio-loud magnetar 1E 1547.0-5408 in April 2022. Unusually however, the source stopped showing radio pulsations ≳ 3 weeks prior to the burst. After recovery, radio timing revealed that the object had also undergone a modest glitch. A model for the overall event is constructed where an initially mild perturbation adjusts the magnetic geometry near the polar caps, leading to shallow fractures. Crustal ejecta or particles leaking from a pair-plasma fireball pollute the magnetospheric gaps, shutting off the pulsar mechanism, but the energy release is not yet large enough to noticeably enhance the X-ray flux. This perturbation gradually ramps, eventuating in a large-scale energy redistribution which fuels the burst. The star's mass quadrupole moment changes in tandem, issuing a glitch. Some quantitative estimates for the magnetic reconfiguration under this interpretation are provided, based on a quasi-static model where the fluid evolves through a

Just need sections

In [18]:
def get_sections(text, indexes): 
    # Looking at sections
    section = {}
    for i in json.loads(text['annotations']['sectionheader']): 
        section['name'] = text['text'][i['start']:i['end']]
        section['start'] = i['start']
        section['end'] = i['end']
        
    return section

In [20]:
get_sections(test)

INTRODUCTION
THEORETICAL OVERVIEW AND RAMPING PERTURBATIONS
HYDROMAGNETIC STRUCTURE
Quadrupolar ellipticity
OUTBURST EVENT IN 1E 1547.0-5408
Shallow fractures: radio switch off
Glitch: quadrupolar shifts
Accelerated spindown
Deeper fractures: X-ray flare
DISCUSSION
APPENDIX A: CALCULATION OF THE ELLIPTICITY AND CRUSTAL STRAIN
A1 Shear modulus
Figure 1 .
Figure 2 .
Figure 3 .


Remove figure and table annotations

Pull relevant texts from DB with right corpusID

In [21]:
temp_df

,index,corpusid,externalids,content,updated
0,0,257219935,"{'arxiv': '2302.13777', 'mag': None, 'acl': No...","{'text': ' The radio shut-off, glitch, and X-r...",2023-06-01T07:32:44Z
1,0,257223374,"{'arxiv': None, 'mag': None, 'acl': None, 'pub...","{'source': {'pdfurls': None, 'pdfsha': 'e1abc5...",2023-06-02T06:11:54Z
2,0,257255588,"{'arxiv': '2303.00647', 'mag': None, 'acl': No...",{'source': {'pdfurls': ['https://export.arxiv....,2023-06-03T15:31:25Z
3,0,257292514,"{'arxiv': None, 'mag': None, 'acl': None, 'pub...","{'source': {'pdfurls': None, 'pdfsha': 'a432bc...",2023-06-02T05:22:08Z
4,0,257297251,"{'arxiv': None, 'mag': None, 'acl': None, 'pub...","{'source': {'pdfurls': None, 'pdfsha': 'eff78e...",2023-05-31T06:25:15Z
